In [1]:
import pandas as pd
import altair as alt
import duckdb

In [4]:
df = pd.read_csv('lobbywatch_medienkonferenz.csv')
df_bezahlt = duckdb.query('select * from df where bezahlt = \'ja\'').to_df()

# Untersuchte Interessenbindungen
Untersucht wurden alle Interessnbindungen, welche folgenden Kriterien entsprechen:
- Sind gemäss Parlamentariergesetz (ParlG) Art. 11 deklarationspflichtig
- Werden von einem gegenwärtig amtierenden Parlamentarier ausgeübt
- Es handelt sich hierbei nicht um eine hauptberufliche Tätigkeit
- Die Interessenbindung wurde gemäss dem Prozess von Lobbywatch recherchiert, kontrolliert und freigegeben
- Die Organisation ist keine Partei, parlamentarische Gruppe, parlamentarische Freundschaftsgruppe oder eine ausserparlamentarische Kommission

# Kennzahlen
Schauen wir uns zuerst ein paar Kennzahlen an

In [8]:
duckdb.query('''
select 'total' kennzahl, (select count(1) from df) wert, 100.0*(select count(1) from df)/(select count(1) from df) anteil
union
select 'deklariert', (select count(1) from df where deklariert = 'ja'), 100.0*(select count(1) from df where deklariert = 'ja')/(select count(1) from df)
union
select 'bezahlt', (select count(1) from df where bezahlt = 'ja'), 100.0*(select count(1) from df where bezahlt = 'ja')/(select count(1) from df)
union
select 'mit angabe zur vergütung', (select count(1) from df where verguetung is not null), 100.0*(select count(1) from df where verguetung is not null)/(select count(1) from df)
union
select 'bezahlt aber undeklariert', (select count(1) from df where bezahlt = 'ja' and deklariert = 'nein'), 100.0*(select count(1) from df where bezahlt = 'ja' and deklariert = 'nein')/(select count(1) from df)
''').to_df()

,kennzahl,wert,anteil
0,total,2340,100.000000
1,deklariert,1749,74.743590
2,bezahlt,838,35.811966
3,mit angabe zur vergütung,253,10.811966
4,bezahlt aber undeklariert,45,1.923077


Bezahlte Interessenbindungen aufgeschlüsselt nach Branche.

In [9]:
source = duckdb.query('''
    select branche, count(1) count
    from df_bezahlt
    group by 1
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl'),
    y=alt.Y('branche', title='Branche'),
    tooltip='count'
)

alt.Chart(...)

In [10]:
source = duckdb.query('''
    select 'total' mass, fraktion, count(1) count
    from df_bezahlt
    group by 1, 2
    union all
    select 'pro parl.', fraktion, (select 1.0*count(1)/(select count(distinct parlamentarier) from df where fraktion = b.fraktion) from df_bezahlt where fraktion = b.fraktion)
    from df_bezahlt b
    group by 1, 2
''').to_df()

alt.Chart(source).mark_bar(opacity=1.0).encode(
    x=alt.X('count', title='Anzahl', stack=None),
    y=alt.Y('fraktion', title='Fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'])),
    color='mass',
    tooltip='count'
)

alt.Chart(...)

Hier sieht man wie sich die Fraktionen die bezahlten Mandate aufteilen innerhalb der Branchen.
Auffällig ist, dass die linken Parteien nur in der Umweltbranche eine Mehrheit der Lobbymandate haben (falls man die GLP zu den linken zählt). Hingegen sind sie in den Branchen Landwirtschaft, Verkehr und Wirtschaft sehr schwach vertreten.

In [11]:
source = duckdb.query('''
    select branche, fraktion, count(1) count
    from df_bezahlt
    where 1=1
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 10)
    group by 1, 2
    order by branche, case fraktion when 'G' then 1 when 'S' then 2 when 'GL' then 3 when 'M-E' then 4 when 'RL' then 5 else 6 end
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', stack='normalize', axis=alt.Axis(format='%')),
    y=alt.Y('branche', title='Branche'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'], range=['#85B50C','#E4022D','#A0A000','#FF9100','#3A8BC1','#0A7228'])),
    order=alt.Order('color_fraktion_sort_index:Q'),
)

alt.Chart(...)

Umgekehrt kann man auch anschauen wo die Schwerpunkte der einzelnen Fraktionen liegen.
_Berücksichtigt hier sind alle Branchen mit insgesamt mindestens 20 bezahlten Lobbymandaten._

In [13]:
source = duckdb.query('''
    select branche, fraktion, count(1) count
    from df_bezahlt
    where branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
    order by branche, case fraktion when 'G' then 1 when 'S' then 2 when 'GL' then 3 when 'M-E' then 4 when 'RL' then 5 else 6 end
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', axis=alt.Axis(format='%'), stack='normalize'),
    y=alt.Y('fraktion', title='Fraktion', sort=['G','S','GL','M-E','RL','V']),
    color='branche',
).properties(
    # height=400
)

alt.Chart(...)

In [14]:
alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', stack='normalize', axis=alt.Axis(format='%')),
    y=alt.Y('branche', title='Branche'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'], range=['#85B50C','#E4022D','#A0A000','#FF9100','#3A8BC1','#0A7228'])),
    order=alt.Order('color_fraktion_sort_index:Q'),
)

alt.Chart(...)

In [15]:
source = duckdb.query('''
    select branche, fraktion, count(1) count
    from df_bezahlt
    where fraktion in (select fraktion from df)
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl'),
    y=alt.Y('fraktion', sort=['G','S','GL','M-E','RL','V'], title='Partei'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'], range=['#85B50C','#E4022D','#A0A000','#FF9100','#3A8BC1','#0A7228'])),
).properties(
    width=250
).facet(
    facet='branche',
    columns=3
)

alt.FacetChart(...)

In [16]:
source = duckdb.query('''
    select branche, fraktion, count(1) count
    from df_bezahlt
    where 1=1
    and fraktion in (select fraktion from df)
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
''').to_df()

alt.Chart(source).mark_arc(innerRadius=40, stroke='white', strokeWidth=1.5, cornerRadius=4).encode(
    theta=alt.Theta('count', stack='normalize'),
    color=alt.Color('branche'),
    tooltip='count',
).properties(
    width=150,
    height=150
).facet(
    facet='fraktion',
    columns=3
)

alt.FacetChart(...)

In [19]:
source = duckdb.query('''
    select distinct p.branche, p.fraktion,
        count(1) over (partition by p.branche, p.fraktion) count,
        count(1) over (partition by p.branche) count_all
    from (
        select branche, fraktion
        from (select distinct fraktion from df_bezahlt) p
        cross join (select branche from df_bezahlt group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df_bezahlt on (p.branche, p.fraktion) = (df_bezahlt.branche, df_bezahlt.fraktion)
    where 1=1
''').to_df()

fraktion_dict = {
    'G': '#85B50C',
    'S': '#E4022D',
    'M-E': '#FF9100',
    'GL': '#A0A000',
    'RL': '#3A8BC1',
    'V': '#0A7228',
}
for p in dict(fraktion_dict).keys():
    if p not in source['fraktion'].values:
        del fraktion_dict[p]

fraktion_namen, fraktion_farben = list(fraktion_dict.keys()), list(fraktion_dict.values())

base = alt.Chart().encode(
    theta=alt.Theta('fraktion', stack=True, scale=alt.Scale(domain=fraktion_namen, rangeMin=0)),
    radius=alt.Radius('rel_count:Q', scale=alt.Scale(type='sqrt', rangeMax=75), stack=False),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
    tooltip=[alt.Tooltip('fraktion'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=180,
    height=180
).transform_calculate(
    rel_count='datum.count / datum.count_all'
)

arcs = base.mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
)

text = base.mark_text(radiusOffset=10).encode(
    text='label:N'
).transform_calculate(
    # label='datum.rel_count == 0 ? "" : format(datum.rel_count, ".1%")'
    label='datum.count == 0 ? "" : datum.count'
)

alt.layer(arcs, text, data=source).facet(
    facet='branche',
    columns=3
).resolve_scale(
    # radius='independent',
    radius='shared',
)

alt.FacetChart(...)

In [20]:
source = duckdb.query('''
    select distinct p.branche, p.partei,
        count(1) over (partition by p.branche, p.partei) count,
        count(1) over (partition by p.branche) count_all
    from (
        select branche, partei
        from (select partei from df group by partei having count(1) >= 10) p
        cross join (select branche from df group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df on (p.branche, p.partei) = (df.branche, df.partei)
    where 1=1
''').to_df()

partei_dict = {
    'Grüne': '#85B50C',
    'SP': '#E4022D',
    'EVP': '#61B5A5',
    'M': '#FF9100',
    'GLP': '#A0A000',
    'FDP': '#3A8BC1',
    'SVP': '#0A7228',
}
for p in dict(partei_dict).keys():
    if p not in source['partei'].values:
        del partei_dict[p]

partei_namen, partei_farben = list(partei_dict.keys()), list(partei_dict.values())

alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4
).encode(
    theta=alt.Theta('branche', stack='center'),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche'),
    tooltip=['branche','count'],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
).facet(
    facet='partei',
    columns=3
).resolve_scale(
    # theta='shared',
    # radius='independent',
)

alt.FacetChart(...)

In [21]:
alt.Chart(source).mark_arc(
    innerRadius=0,
    # theta2=math.pi/4,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4
).encode(
    theta=alt.Theta('count', stack=None),
    radius=alt.Radius('branche:N', scale=alt.Scale(type='identity', rangeMin=10, rangeMax=70), stack='center'),
    color=alt.Color('branche'),
    tooltip=['branche','count'],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
).facet(
    facet='partei',
    columns=3
).resolve_scale(
    theta='independent',
    # radius='independent',
)

alt.FacetChart(...)

In [23]:
source = duckdb.query('''
    select distinct p.branche, p.parlamentarier, df.partei, df.kanton,
        count(1) over (partition by p.branche, p.parlamentarier) count,
        count(1) over (partition by p.branche) count_all
    from (
        select branche, parlamentarier
        from (select parlamentarier as parlamentarier from df group by 1 order by count(1) desc limit 12) p
        cross join (select branche from df group by branche order by count(1) desc limit 8) b
    ) p left join df on (p.branche, p.parlamentarier) = (df.branche, df.parlamentarier)
    where 1=1
    and df.partei is not null and df.kanton is not null
''').to_df()

alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
).encode(
    theta=alt.Theta('branche', stack='center', scale=alt.Scale(rangeMin=0)),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche'),
    tooltip=[alt.Tooltip('branche'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all',
    parl_label='datum.parlamentarier + " (" + datum.partei + "/" + datum.kanton + ")"'
).facet(
    facet='parl_label:N',
    columns=3
).resolve_scale(
    # radius='independent',
    # radius='shared',
)

alt.FacetChart(...)

In [24]:
df

,parlamentarier,parlamentarier_id,partei,fraktion,kanton,geschlecht,interessenbindung_id,deklariert,art,funktion,wirksamkeit,seit,bezahlt,verguetung,organisation,uid,branche,lobbygruppe,rechtsform
0,Ada Marra,3923,SP,S,VD,F,7111,ja,beirat,mitglied,tief,NaN,nein,NaN,Fondation Marie-Eléonore d'Olcah,CHE-443.938.715,Kultur,Kulturinstitutionen,Stiftung
1,Ada Marra,3923,SP,S,VD,F,7161,ja,vorstand,praesident,mittel,NaN,nein,NaN,Nationale Plattform zu den Sans-Papiers,NaN,Soziale Sicherheit,Gemeinwohl/Gesellschaft,Informelle Gruppe
2,Ada Marra,3923,SP,S,VD,F,13185,ja,vorstand,praesident,tief,2022-01-01,nein,NaN,Association vaudoise pour la sauvegarde des lo...,NaN,Soziale Sicherheit,Gemeinwohl/Gesellschaft,Verein
3,Adèle Thorens Goumaz,3907,Grüne,G,VD,F,12318,ja,taetig,NaN,tief,2021-11-02,ja,NaN,sanu Future Learning AG,CHE-421.061.771,Bildung,Bildung/Wissenschaft,AG
4,Adèle Thorens Goumaz,3907,Grüne,G,VD,F,12133,ja,beirat,NaN,tief,2021-09-03,ja,NaN,IMD Business School,NaN,Bildung,Bildung/Wissenschaft,Einzelunternehmen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,Yvonne Feri,4069,SP,S,AG,F,6103,nein,beirat,mitglied,tief,2017-01-18,nein,NaN,Verein Projekt Doppeltür,CHE-138.494.131,Staatspolitik/Staatswirtschaft,Religion,Verein
2336,Yvonne Feri,4069,SP,S,AG,F,13470,nein,vorstand,mitglied,tief,2021-09-29,nein,NaN,Stiftung zur Unterstützung von Einelternfamili...,CHE-112.124.916,Soziale Sicherheit,Gemeinwohl/Gesellschaft,Stiftung
2337,Yvonne Feri,4069,SP,S,AG,F,13438,nein,geschaeftsfuehrend,praesident,tief,2022-03-31,NaN,NaN,Kampahire GmbH,CHE-373.444.446,Wirtschaft,Dienstleistungen allg.,GmbH
2338,Yvonne Feri,4069,SP,S,AG,F,2638,nein,beirat,mitglied,mittel,NaN,nein,NaN,Stiftung für junge Auslandschweizer,CHE-101.334.438,Soziale Sicherheit,Kinder/Jugend,Stiftung


In [39]:
source = duckdb.query('''
select fraktion, branche, (select count(1) from df where branche = b.branche and fraktion = f.fraktion) count
from (select distinct fraktion from df_bezahlt) as f
cross join (select distinct branche from df_bezahlt) as b
group by 1, 2
''').to_df()

alt.Chart(source).mark_rect().encode(
    x='branche',
    y='fraktion',
    tooltip='count',
    color=alt.Color('count',
        scale=alt.Scale(scheme='yelloworangered', domain=[0,150]),
        legend=alt.Legend(direction='horizontal')
    )
)

alt.Chart(...)